# Week 3/4

### Sensitivity Curve Interactive Dashboard  
Type in values for parameters and click 'Run Interact' to generate corresponding sensitivity curve. Click the legend to hide curves. NOTE: "half-maximum" values are calculated using findpeaks on negative sensitivity curve. CAUTION: Changing itmT changes the optimal prmT (for impedance matching) and find_optimal_prmT will execute to optimise prmT (slow). TODO: Error catching for unexpected input; speed up code.  
14/12/22: Updated to plotly plots for interactivity. Use negative sensitivity to preserve shape.

In [13]:
import finesse
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

itmT_store = -1  # Only optimise prmT if itmT changes

def plot(diff,comm,srcL,vsrmL,vsrmT,itmT): # This executes when 'Interact' is clicked
    global itmT_store
    
    # Initialise variables (
    # if no peak is found a log error will be thrown for the maximum and half-maximum lines but the sensitivity curves are OK)
    fsig = np.geomspace(500,10e3,201)
    peak_sens = 0
    peak_f = 0
    peak_bw = 0
    left_f = 0
    right_f = 0
    
    diff = float(diff)
    comm = float(comm)
    srcL = float(srcL)
    vsrmL = float(vsrmL)
    vsrmT = float(vsrmT)
    itmT = float(itmT)
    prmT = 0
    
    if itmT != 0.01397:
        if itmT != itmT_store:
            peak_power, prmT = find_optimal_prmT(itmT)
            print(f"Peak Arm Power: {peak_power}W when prmT: {prmT}")
            itmT_store = itmT
        
    else:
        prmT = 0.03
    
    kat = finesse.Model()
    # From SRC_tunability_DCreadout_changedBW_fin3exp.ipynb
    kat.parse(
        f"""
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1
    var itmT {itmT}
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 500
    
    # CHECK Input laser power
    pd P_in L0.p1.o
    
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45
    
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx
    
    m itmxar T=1 L=0 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm
    
    m etmx T=5u L=40u phi=179.99999
    
    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy
    
    m itmyar T=1 L=0 phi=90
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm
    
    m etmy T=5u L=40u phi=90.00001
    
    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i
    
    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M
    
    ###########################################################################
    ###   vSRM
    ###########################################################################
    
    #m srm T=0.2 L=37.5u phi=-90
    
    s src bs.p4 SRC_BS.p1 L={srcL}
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L={vsrmL}
    m vSRM1 T={vsrmT} L=0 phi={-90+comm+diff}
    s vSRC2  SRC_BS.p3 vSRM2.p1 L={vsrmL}
    m vSRM2 T={vsrmT} L=0 phi={0+comm-diff}
    
    # CHECK SRC power
    pd P_SRC SRC_BS.p1.i
   
    ###########################################################################
    ###   Output & squeezing
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    #sq sqz db=-10 angle=90
    #link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    qshot NSR_without_RP AS.p1.i nsr=True
    pd1 signal AS.p1.i f=fsig
    ad trans SRC_BS.p4.o f=fsig

    fsig(1)
    xaxis(fsig, log, 500, 10k, 200)
    
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP']) # Take reciprocal sensitivity to use findpeaks i.e. FWHM defined by using reciprocal sensitivity!
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size != 0: # If a peak is found (first peak taken)
        fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
        left_idx = fwhm_idxs[2][0]
        right_idx = fwhm_idxs[3][0]
        interp_fsig = interpolate.interp1d(np.arange(201), fsig)
        
        left_f = interp_fsig(left_idx)
        right_f = interp_fsig(right_idx)
        peak_sens = np.real(out['NSR_with_RP'])[peak_idxs[0]]
        peak_f = fsig[peak_idxs[0]]
        peak_bw = right_f - left_f

    
    fig_qnoise = go.Figure()
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_with_RP']),mode='lines+markers',name='qnoised NSR'))
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_without_RP']),mode='lines+markers',name='qshot NSR'))
    fig_qnoise.update_xaxes(type="log")
    fig_qnoise.update_yaxes(type="log")
    fig_qnoise.add_vline(x=peak_f)
    fig_qnoise.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_qnoise.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_qnoise.update_layout(title="Sensitivity (qnoised, qshot)",xaxis_title="Frequency [Hz]",yaxis_title="Sensitivity [1/rt Hz]")
    fig_qnoise.show()
    
    fig_signal = go.Figure()
    fig_signal.add_trace(go.Scatter(x=fsig, y=np.abs(out['signal']),mode='lines+markers'))
    fig_signal.update_xaxes(type="log")
    fig_signal.update_yaxes(type="log")
    fig_signal.add_vline(x=peak_f)
    fig_signal.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_signal.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_signal.update_layout(title="Signal Gain (pd1)",xaxis_title="Frequency [Hz]",yaxis_title="Power [W]")
    fig_signal.show()
    
    # Print model outputs
    print(f"Peak Sensitivity: {peak_sens} 1/rt Hz, Peak Frequency: {peak_f}Hz, Peak FWHM: {peak_bw}Hz")
    print(f"Input laser power: {np.max(np.abs(out['P_in']))}W")
    print(f"PRC power: {np.max(np.abs(out['P_PRC']))*1e-3}kW")
    print(f"Laser power incident on BS: {np.max(np.abs(out['P_BS']))*1e-3}kW")
    print(f"X-arm cavity power: {np.max(np.abs(out['P_armX']))*1e-6}MW")
    print(f"Y-arm cavity power: {np.max(np.abs(out['P_armY']))*1e-6}MW")
    print(f"SRC power: {np.max(np.abs(out['P_SRC']))}W")

# Default values here
widgets.interact_manual(plot, diff=widgets.Text(value='6.3565'),
                 comm=widgets.Text(value='0'),
                 srcL=widgets.Text(value='354'),
                 vsrmL=widgets.Text(value='0'),
                 vsrmT=widgets.Text(value='0'),
                 itmT=widgets.Text(value='0.01397'));

# Auto-tune prmT with findpeaks to maximise arm cavity power (impedance matching)
def find_optimal_prmT(itmT):
    vary_prmT = np.linspace(0.001,0.999,201)
    circX = np.zeros((201,))

    for i, prmT in enumerate(vary_prmT):
        kat = finesse.Model()
        kat.parse(
            f"""
        ###########################################################################
        ###   Variables
        ###########################################################################
        var Larm 4000
        var Mtm  74.1
        var itmT {itmT}
        var lmichx 4.5
        var lmichy 4.45

        ###########################################################################
        ###   Input optics
        ###########################################################################
        l L0 500
        s l_in L0.p1 prm.p1
        # Power recycling mirror
        m prm T={prmT} L=2e-05 phi=90
        s prc prm.p2 bs.p1 L=53

        # Central beamsplitter
        bs bs T=0.5 L=0 alpha=45

        ###########################################################################
        ###   X arm
        ###########################################################################
        s lx bs.p3 itmxar.p1 L=lmichx

        m itmxar T=1 L=0 phi=180
        s ar_thick itmxar.p2 itmx.p1 L=0
        m itmx T=itmT L=20u phi=180
        s LX itmx.p2 etmx.p1 L=Larm

        m etmx T=5u L=40u phi=179.99999

        # CHECK X-arm cavity power
        pd P_armX etmx.p1.i

        pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
        pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

        ###########################################################################
        ###   Y arm
        ###########################################################################
        s ly bs.p2 itmyar.p1 L=lmichy

        m itmyar T=1 L=0 phi=90
        s ar_thicky itmyar.p2 itmy.p1 L=0
        m itmy T=itmT L=20u phi=90
        s LY itmy.p2 etmy.p1 L=Larm

        m etmy T=5u L=40u phi=90.00001
        
        # CHECK Y-arm cavity power
        pd P_armY etmy.p1.i

        pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
        pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

        ###########################################################################
        ###   vSRM
        ###########################################################################

        s src bs.p4 SRC_BS.p1 L=354
        bs SRC_BS T=0.5 L=0 alpha=45
        s vSRC1 SRC_BS.p2 vSRM1.p1 L=0
        m vSRM1 T=0 L=0 phi=-83.6435
        s vSRC2 SRC_BS.p3 vSRM2.p1 L=0
        m vSRM2 T=0 L=0 phi=-6.3565
        
        noxaxis()

        """
        )
        out = kat.run()
        circX[i] = out['P_armX']

    peak_power = circX[np.argmax(circX)]
    peak_T = vary_prmT[np.argmax(circX)]
    return peak_power, peak_T


interactive(children=(Text(value='6.3565', description='diff'), Text(value='0', description='comm'), Text(valu…

### Effects of Single-Parameter Variation on Sensitivity  
For the parameters: differential mode tuning $\phi_\text{diff}$, common mode tuning $\phi_\text{comm}$, SRC length $L_\text{SRC}$, vSRM arm length $L_\text{vSRM}$, vSRM transission $T_\text{vSRM}$, ITM transmission $T_\text{ITM}$, findpeaks was used to find the peak in the negative sensitivity curve. We show the variation of the peak frequency, peak sensitivity, and peak FWHM (the half-maximum again measured with respect to the negative sensitivity curve). If no peak was found, the minimum in the sensitivity curve is reported (the frequency range is 500Hz - 10kHz). An optimal detector will have high sensitivity (low on graph) at high frequencies (kHz) with high FWHM.
TODO: Narrow in on optimal parameter ranges and increase resolution. Compare results when findpeaks is used on RECIPROCAL sensitivity. 

**Figure 1a:** Peak $\text{ASD}$ vs. $\phi_\text{diff}$  
**Figure 1b:** Peak $f$ vs. $\phi_\text{diff}$  
**Figure 1c:** Peak $\text{FWHM}$ vs. $\phi_\text{diff}$  
NOTE: Using findpeaks on reciprocal sensitivity leads to a noticeable difference in peak bandwidth to negative sensitivity.

In [2]:
import finesse
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

fsig = np.geomspace(500,10e3,201)
vary_diff = np.linspace(-10, 10, 201)
peak_sens = np.zeros((201,))
peak_f = np.zeros((201,))
peak_bw = np.zeros((201,))
nopeak_sens = np.zeros((201,))
nopeak_f = np.zeros((201,))

# Vary parameter, measure peak sensitivity, peak frequency, peak FWHM (bandwidth)
for i, diff in enumerate(vary_diff):
    kat = finesse.Model()
    kat.parse(
        f"""
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1
    var itmT 0.01397
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 500
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T=0.03 L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45
    
    # Check power in PRC
    #pd reflPRC prm.p1.o
    #pd circPRC bs.p1.i
    #pd trnsPRC bs.p3.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx
    
    m itmxar T=1 L=0 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm
    
    m etmx T=5u L=40u phi=179.99999
    
    # Check power in ARM X cavity
    #pd reflX itmx.p1.o
    #pd circX etmx.p1.i
    #pd trnsX etmx.p2.o

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy
    
    m itmyar T=1 L=0 phi=90
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm
    
    m etmy T=5u L=40u phi=90.00001
    
    # Check power in ARM Y cavity
    #pd reflY itmy.p1.o
    #pd circY etmy.p1.i
    #pd trnsY etmy.p2.o

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M
    
    ###########################################################################
    ###   vSRM
    ###########################################################################
    
    #m srm T=0.2 L=37.5u phi=-90
    
    s src bs.p4 SRC_BS.p1 L=354
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L=0
    m vSRM1 T=0 L=0 phi={-90+diff}
    s vSRC2  SRC_BS.p3 vSRM2.p1 L=0
    m vSRM2 T=0 L=0 phi={0-diff}
    
    # Check power in SRC
    #pd reflSRC bs.p2.o
    #pd circSRC SRC_BS.p1.i
    #pd trnsSRC SRC_BS.p3.o
   
    ###########################################################################
    ###   Output & squeezing
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    #sq sqz db=-10 angle=90
    #link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    #qshot NSR_without_RP AS.p1.i nsr=True
    #pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 500, 10k, 200)
    
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP'])
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size == 0: # If no peak found, the max sensitivity and its frequency (lower bound is 500Hz) is recorded
        nopeak_sens[i] = np.min(np.abs(out['NSR_with_RP']))
        nopeak_f[i] = fsig[np.argmin(np.abs(out['NSR_with_RP']))]
        peak_sens[i] = np.nan
        peak_f[i] = np.nan
        peak_bw[i] = np.nan
        continue
    else:
        nopeak_sens[i] = np.nan
        nopeak_f[i] = np.nan
        
    fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
    left_idx = fwhm_idxs[2][0]
    right_idx = fwhm_idxs[3][0]
    interp_fsig = interpolate.interp1d(np.arange(201), fsig)
    peak_sens[i] = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
    peak_f[i] = fsig[peak_idxs[0]]
    peak_bw[i] = interp_fsig(right_idx) - interp_fsig(left_idx)
    
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_diff, y=peak_sens,mode='lines+markers',name='Peak'))
fig_peak_sens.add_trace(go.Scatter(x=vary_diff, y=nopeak_sens,mode='lines+markers',name='No peak'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. Differential Mode Tuning",xaxis_title="Differential Mode Tuning [deg]",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_diff, y=peak_f,mode='lines+markers',name='Peak'))
fig_peak_f.add_trace(go.Scatter(x=vary_diff, y=nopeak_f,mode='lines+markers',name='No peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. Differential Mode Tuning",xaxis_title="Differential Mode Tuning [deg]",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = px.line(x=vary_diff, y=peak_bw, markers=True, title="Peak Bandwidth vs. Differential Mode Tuning")
fig_peak_bw.update_xaxes(title_text="Differential Mode Tuning [deg]")
fig_peak_bw.update_yaxes(title_text="Peak Bandwidth [Hz]")
fig_peak_bw.show()

**Figure 2a:** Peak $\text{ASD}$ vs. $\phi_\text{comm}$  
**Figure 2b:** Peak $f$ vs. $\phi_\text{comm}$  
**Figure 2c:** Peak $\text{FWHM}$ vs. $\phi_\text{comm}$  
NOTE: The first peak found is the one recorded! The default 6.3565$^\circ$ differential mode tuning is used.

In [3]:
import finesse
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

fsig = np.geomspace(500,10e3,201)
vary_comm = np.linspace(-10, 10, 201)
peak_sens = np.zeros((201,))
peak_f = np.zeros((201,))
peak_bw = np.zeros((201,))
nopeak_sens = np.zeros((201,))
nopeak_f = np.zeros((201,))

# Vary parameter, measure peak sensitivity, peak frequency, peak FWHM (bandwidth)
for i, comm in enumerate(vary_comm):
    kat = finesse.Model()
    kat.parse(
        f"""
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1
    var itmT 0.01397
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 500
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T=0.03 L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45
    
    # Check power in PRC
    #pd reflPRC prm.p1.o
    #pd circPRC bs.p1.i
    #pd trnsPRC bs.p3.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx
    
    m itmxar T=1 L=0 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm
    
    m etmx T=5u L=40u phi=179.99999
    
    # Check power in ARM X cavity
    #pd reflX itmx.p1.o
    #pd circX etmx.p1.i
    #pd trnsX etmx.p2.o

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy
    
    m itmyar T=1 L=0 phi=90
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm
    
    m etmy T=5u L=40u phi=90.00001
    
    # Check power in ARM Y cavity
    #pd reflY itmy.p1.o
    #pd circY etmy.p1.i
    #pd trnsY etmy.p2.o

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M
    
    ###########################################################################
    ###   vSRM
    ###########################################################################
    
    #m srm T=0.2 L=37.5u phi=-90
    
    s src bs.p4 SRC_BS.p1 L=354
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L=0
    m vSRM1 T=0 L=0 phi={-83.6435+comm}
    s vSRC2  SRC_BS.p3 vSRM2.p1 L=0
    m vSRM2 T=0 L=0 phi={-6.3565+comm}
    
    # Check power in SRC
    #pd reflSRC bs.p2.o
    #pd circSRC SRC_BS.p1.i
    #pd trnsSRC SRC_BS.p3.o
   
    ###########################################################################
    ###   Output & squeezing
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    #sq sqz db=-10 angle=90
    #link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    #qshot NSR_without_RP AS.p1.i nsr=True
    #pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 500, 10k, 200)
    
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP'])
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size == 0: # If no peak found, the max sensitivity and its frequency (lower bound is 500Hz) is recorded
        nopeak_sens[i] = np.min(np.abs(out['NSR_with_RP']))
        nopeak_f[i] = fsig[np.argmin(np.abs(out['NSR_with_RP']))]
        peak_sens[i] = np.nan
        peak_f[i] = np.nan
        peak_bw[i] = np.nan
        continue
    else:
        nopeak_sens[i] = np.nan
        nopeak_f[i] = np.nan
        
    fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
    left_idx = fwhm_idxs[2][0]
    right_idx = fwhm_idxs[3][0]
    interp_fsig = interpolate.interp1d(np.arange(201), fsig)
    peak_sens[i] = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
    peak_f[i] = fsig[peak_idxs[0]]
    peak_bw[i] = interp_fsig(right_idx) - interp_fsig(left_idx)
    
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_comm, y=peak_sens,mode='lines+markers',name='Peak'))
fig_peak_sens.add_trace(go.Scatter(x=vary_comm, y=nopeak_sens,mode='lines+markers',name='No peak'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. Common Mode Tuning",xaxis_title="Common Mode Tuning [deg]",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_comm, y=peak_f,mode='lines+markers',name='Peak'))
fig_peak_f.add_trace(go.Scatter(x=vary_comm, y=nopeak_f,mode='lines+markers',name='No peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. Common Mode Tuning",xaxis_title="Common Mode Tuning [deg]",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = px.line(x=vary_comm, y=peak_bw, markers=True, title="Peak Bandwidth vs. Common Mode Tuning")
fig_peak_bw.update_xaxes(title_text="Common Mode Tuning [deg]")
fig_peak_bw.update_yaxes(title_text="Peak Bandwidth [Hz]")
fig_peak_bw.show()

**Figure 3a:** Peak $\text{ASD}$ vs. $L_\text{SRC}$  
**Figure 3b:** Peak $f$ vs. $L_\text{SRC}$  
**Figure 3c:** Peak $\text{FWHM}$ vs. $L_\text{SRC}$

In [4]:
import finesse
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

fsig = np.geomspace(500,10e3,201)
vary_srcL = np.linspace(10, 1010, 201)
peak_sens = np.zeros((201,))
peak_f = np.zeros((201,))
peak_bw = np.zeros((201,))
nopeak_sens = np.zeros((201,))
nopeak_f = np.zeros((201,))

# Vary parameter, measure peak sensitivity, peak frequency, peak FWHM (bandwidth)
for i, srcL in enumerate(vary_srcL):
    kat = finesse.Model()
    kat.parse(
        f"""
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1
    var itmT 0.01397
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 500
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T=0.03 L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45
    
    # Check power in PRC
    #pd reflPRC prm.p1.o
    #pd circPRC bs.p1.i
    #pd trnsPRC bs.p3.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx
    
    m itmxar T=1 L=0 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm
    
    m etmx T=5u L=40u phi=179.99999
    
    # Check power in ARM X cavity
    #pd reflX itmx.p1.o
    #pd circX etmx.p1.i
    #pd trnsX etmx.p2.o

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy
    
    m itmyar T=1 L=0 phi=90
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm
    
    m etmy T=5u L=40u phi=90.00001
    
    # Check power in ARM Y cavity
    #pd reflY itmy.p1.o
    #pd circY etmy.p1.i
    #pd trnsY etmy.p2.o

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M
    
    ###########################################################################
    ###   vSRM
    ###########################################################################
    
    #m srm T=0.2 L=37.5u phi=-90
    
    s src bs.p4 SRC_BS.p1 L={srcL}
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L=0
    m vSRM1 T=0 L=0 phi=-83.6435
    s vSRC2  SRC_BS.p3 vSRM2.p1 L=0
    m vSRM2 T=0 L=0 phi=-6.3565
    
    # Check power in SRC
    #pd reflSRC bs.p2.o
    #pd circSRC SRC_BS.p1.i
    #pd trnsSRC SRC_BS.p3.o
   
    ###########################################################################
    ###   Output & squeezing
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    #sq sqz db=-10 angle=90
    #link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    #qshot NSR_without_RP AS.p1.i nsr=True
    #pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 500, 10k, 200)
    
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP'])
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size == 0: # If no peak found, the max sensitivity and its frequency (lower bound is 500Hz) is recorded
        nopeak_sens[i] = np.min(np.abs(out['NSR_with_RP']))
        nopeak_f[i] = fsig[np.argmin(np.abs(out['NSR_with_RP']))]
        peak_sens[i] = np.nan
        peak_f[i] = np.nan
        peak_bw[i] = np.nan
        continue
    else:
        nopeak_sens[i] = np.nan
        nopeak_f[i] = np.nan
        
    fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
    left_idx = fwhm_idxs[2][0]
    right_idx = fwhm_idxs[3][0]
    interp_fsig = interpolate.interp1d(np.arange(201), fsig)
    peak_sens[i] = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
    peak_f[i] = fsig[peak_idxs[0]]
    peak_bw[i] = interp_fsig(right_idx) - interp_fsig(left_idx)
    
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_srcL, y=peak_sens,mode='lines+markers',name='Peak'))
fig_peak_sens.add_trace(go.Scatter(x=vary_srcL, y=nopeak_sens,mode='lines+markers',name='No peak'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. SRC Length",xaxis_title="SRC Length [m]",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_srcL, y=peak_f,mode='lines+markers',name='Peak'))
fig_peak_f.add_trace(go.Scatter(x=vary_srcL, y=nopeak_f,mode='lines+markers',name='No peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. SRC Length",xaxis_title="SRC Length [m]",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = px.line(x=vary_srcL, y=peak_bw, markers=True, title="Peak Bandwidth vs. SRC Length")
fig_peak_bw.update_xaxes(title_text="SRC Length [m]")
fig_peak_bw.update_yaxes(title_text="Peak Bandwidth [Hz]")
fig_peak_bw.show()

**Figure 4a:** Peak $\text{ASD}$ vs. $L_\text{vSRM}$  
**Figure 4b:** Peak $f$ vs. $L_\text{vSRM}$  
**Figure 4c:** Peak $\text{FWHM}$ vs. $L_\text{vSRM}$  
TODO: Investigate anomalous stratification in Figure 4b.

In [5]:
import finesse
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

fsig = np.geomspace(500,10e3,201)
vary_vsrmL = np.linspace(1,101,201)
peak_sens = np.zeros((201,))
peak_f = np.zeros((201,))
peak_bw = np.zeros((201,))
nopeak_sens = np.zeros((201,))
nopeak_f = np.zeros((201,))

# Vary parameter, measure peak sensitivity, peak frequency, peak FWHM (bandwidth)
for i, vsrmL in enumerate(vary_vsrmL):
    kat = finesse.Model()
    kat.parse(
        f"""
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1
    var itmT 0.01397
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 500
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T=0.03 L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45
    
    # Check power in PRC
    #pd reflPRC prm.p1.o
    #pd circPRC bs.p1.i
    #pd trnsPRC bs.p3.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx
    
    m itmxar T=1 L=0 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm
    
    m etmx T=5u L=40u phi=179.99999
    
    # Check power in ARM X cavity
    #pd reflX itmx.p1.o
    #pd circX etmx.p1.i
    #pd trnsX etmx.p2.o

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy
    
    m itmyar T=1 L=0 phi=90
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm
    
    m etmy T=5u L=40u phi=90.00001
    
    # Check power in ARM Y cavity
    #pd reflY itmy.p1.o
    #pd circY etmy.p1.i
    #pd trnsY etmy.p2.o

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M
    
    ###########################################################################
    ###   vSRM
    ###########################################################################
    
    #m srm T=0.2 L=37.5u phi=-90
    
    s src bs.p4 SRC_BS.p1 L=354
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L={vsrmL}
    m vSRM1 T=0 L=0 phi=-83.6435
    s vSRC2  SRC_BS.p3 vSRM2.p1 L={vsrmL}
    m vSRM2 T=0 L=0 phi=-6.3565
    
    # Check power in SRC
    #pd reflSRC bs.p2.o
    #pd circSRC SRC_BS.p1.i
    #pd trnsSRC SRC_BS.p3.o
   
    ###########################################################################
    ###   Output & squeezing
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    #sq sqz db=-10 angle=90
    #link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    #qshot NSR_without_RP AS.p1.i nsr=True
    #pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 500, 10k, 200)
    
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP'])
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size == 0: # If no peak found, the max sensitivity and its frequency (lower bound is 500Hz) is recorded
        nopeak_sens[i] = np.min(np.abs(out['NSR_with_RP']))
        nopeak_f[i] = fsig[np.argmin(np.abs(out['NSR_with_RP']))]
        peak_sens[i] = np.nan
        peak_f[i] = np.nan
        peak_bw[i] = np.nan
        continue
    else:
        nopeak_sens[i] = np.nan
        nopeak_f[i] = np.nan
        
    fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
    left_idx = fwhm_idxs[2][0]
    right_idx = fwhm_idxs[3][0]
    interp_fsig = interpolate.interp1d(np.arange(201), fsig)
    peak_sens[i] = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
    peak_f[i] = fsig[peak_idxs[0]]
    peak_bw[i] = interp_fsig(right_idx) - interp_fsig(left_idx)
    
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_vsrmL, y=peak_sens,mode='lines+markers',name='Peak'))
fig_peak_sens.add_trace(go.Scatter(x=vary_vsrmL, y=nopeak_sens,mode='lines+markers',name='No peak'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. vSRM Arm Length",xaxis_title="vSRM Arm Length [m]",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_vsrmL, y=peak_f,mode='lines+markers',name='Peak'))
fig_peak_f.add_trace(go.Scatter(x=vary_vsrmL, y=nopeak_f,mode='lines+markers',name='No peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. vSRM Arm Length",xaxis_title="vSRM Arm Length [m]",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = px.line(x=vary_vsrmL, y=peak_bw, markers=True, title="Peak Bandwidth vs. vSRM Arm Length")
fig_peak_bw.update_xaxes(title_text="vSRM Arm Length [m]")
fig_peak_bw.update_yaxes(title_text="Peak Bandwidth [Hz]")
fig_peak_bw.show()

**Figure 5a:** Peak $\text{ASD}$ vs. $T_\text{vSRM}$  
**Figure 5b:** Peak $f$ vs. $T_\text{vSRM}$  
**Figure 5c:** Peak $\text{FWHM}$ vs. $T_\text{vSRM}$

In [6]:
import finesse
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

fsig = np.geomspace(500,10e3,201)
vary_vsrmT = np.linspace(0.001,0.201,201)
peak_sens = np.zeros((201,))
peak_f = np.zeros((201,))
peak_bw = np.zeros((201,))
nopeak_sens = np.zeros((201,))
nopeak_f = np.zeros((201,))

# Vary parameter, measure peak sensitivity, peak frequency, peak FWHM (bandwidth)
for i, vsrmT in enumerate(vary_vsrmT):
    kat = finesse.Model()
    kat.parse(
        f"""
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1
    var itmT 0.01397
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 500
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T=0.03 L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45
    
    # Check power in PRC
    #pd reflPRC prm.p1.o
    #pd circPRC bs.p1.i
    #pd trnsPRC bs.p3.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx
    
    m itmxar T=1 L=0 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm
    
    m etmx T=5u L=40u phi=179.99999
    
    # Check power in ARM X cavity
    #pd reflX itmx.p1.o
    #pd circX etmx.p1.i
    #pd trnsX etmx.p2.o

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy
    
    m itmyar T=1 L=0 phi=90
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm
    
    m etmy T=5u L=40u phi=90.00001
    
    # Check power in ARM Y cavity
    #pd reflY itmy.p1.o
    #pd circY etmy.p1.i
    #pd trnsY etmy.p2.o

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M
    
    ###########################################################################
    ###   vSRM
    ###########################################################################
    
    #m srm T=0.2 L=37.5u phi=-90
    
    s src bs.p4 SRC_BS.p1 L=354
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L=0
    m vSRM1 T={vsrmT} L=0 phi=-83.6435
    s vSRC2  SRC_BS.p3 vSRM2.p1 L=0
    m vSRM2 T={vsrmT} L=0 phi=-6.3565
    
    # Check power in SRC
    #pd reflSRC bs.p2.o
    #pd circSRC SRC_BS.p1.i
    #pd trnsSRC SRC_BS.p3.o
   
    ###########################################################################
    ###   Output & squeezing
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    #sq sqz db=-10 angle=90
    #link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    #qshot NSR_without_RP AS.p1.i nsr=True
    #pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 500, 10k, 200)
    
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP'])
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size == 0: # If no peak found, the max sensitivity and its frequency (lower bound is 500Hz) is recorded
        nopeak_sens[i] = np.min(np.abs(out['NSR_with_RP']))
        nopeak_f[i] = fsig[np.argmin(np.abs(out['NSR_with_RP']))]
        peak_sens[i] = np.nan
        peak_f[i] = np.nan
        peak_bw[i] = np.nan
        continue
    else:
        nopeak_sens[i] = np.nan
        nopeak_f[i] = np.nan
        
    fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
    left_idx = fwhm_idxs[2][0]
    right_idx = fwhm_idxs[3][0]
    interp_fsig = interpolate.interp1d(np.arange(201), fsig)
    peak_sens[i] = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
    peak_f[i] = fsig[peak_idxs[0]]
    peak_bw[i] = interp_fsig(right_idx) - interp_fsig(left_idx)
    
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_vsrmT, y=peak_sens,mode='lines+markers',name='Peak'))
fig_peak_sens.add_trace(go.Scatter(x=vary_vsrmT, y=nopeak_sens,mode='lines+markers',name='No peak'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. vSRM T",xaxis_title="vSRM T",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_vsrmT, y=peak_f,mode='lines+markers',name='Peak'))
fig_peak_f.add_trace(go.Scatter(x=vary_vsrmT, y=nopeak_f,mode='lines+markers',name='No peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. vSRM T",xaxis_title="vSRM T",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = px.line(x=vary_vsrmT, y=peak_bw, markers=True, title="Peak Bandwidth vs. vSRM T")
fig_peak_bw.update_xaxes(title_text="vSRM T")
fig_peak_bw.update_yaxes(title_text="Peak Bandwidth [Hz]")
fig_peak_bw.show()

**Figure 6a:** Peak $\text{ASD}$ vs. $T_\text{ITM}$  
**Figure 6b:** Peak $f$ vs. $T_\text{ITM}$  
**Figure 6c:** Peak $\text{FWHM}$ vs. $T_\text{ITM}$  
TODO: Make this code more efficient!

In [7]:
import finesse
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

def find_optimal_prmT(itmT):  # Hold all other parameters constant when varying itmT
    vary_prmT = np.linspace(0,0.05,51)
    circX = np.zeros((51,))  # Assume that power in arms is equal (have checked this) TODO: build in the check

    for i, prmT in enumerate(vary_prmT):
        kat = finesse.Model()
        kat.parse(
            f"""
        ###########################################################################
        ###   Variables
        ###########################################################################
        var Larm 4000
        var Mtm  74.1
        var itmT {itmT}
        var lmichx 4.5
        var lmichy 4.45

        ###########################################################################
        ###   Input optics
        ###########################################################################
        l L0 500
        s l_in L0.p1 prm.p1
        # Power recycling mirror
        m prm T={prmT} L=2e-05 phi=90
        s prc prm.p2 bs.p1 L=53

        # Central beamsplitter
        bs bs T=0.5 L=0 alpha=45

        ###########################################################################
        ###   X arm
        ###########################################################################
        s lx bs.p3 itmxar.p1 L=lmichx

        m itmxar T=1 L=0 phi=180
        s ar_thick itmxar.p2 itmx.p1 L=0
        m itmx T=itmT L=20u phi=180
        s LX itmx.p2 etmx.p1 L=Larm

        m etmx T=5u L=40u phi=179.99999

        # CHECK X-arm cavity power
        pd P_armX etmx.p1.i

        pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
        pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

        ###########################################################################
        ###   Y arm
        ###########################################################################
        s ly bs.p2 itmyar.p1 L=lmichy

        m itmyar T=1 L=0 phi=90
        s ar_thicky itmyar.p2 itmy.p1 L=0
        m itmy T=itmT L=20u phi=90
        s LY itmy.p2 etmy.p1 L=Larm

        m etmy T=5u L=40u phi=90.00001
        
        # CHECK Y-arm cavity power
        pd P_armY etmy.p1.i

        pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
        pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

        ###########################################################################
        ###   vSRM
        ###########################################################################

        s src bs.p4 SRC_BS.p1 L=354
        bs SRC_BS T=0.5 L=0 alpha=45
        s vSRC1 SRC_BS.p2 vSRM1.p1 L=0
        m vSRM1 T=0 L=0 phi=-83.6435
        s vSRC2 SRC_BS.p3 vSRM2.p1 L=0
        m vSRM2 T=0 L=0 phi=-6.3565
        
        noxaxis()

        """
        )
        out = kat.run()
        circX[i] = out['P_armX']
        
    peak_power = circX[np.argmax(circX)] # Maximimise power in the arms
    peak_T = vary_prmT[np.argmax(circX)]
    return peak_power, peak_T

fsig = np.geomspace(500,10e3,201)
vary_itmT = np.linspace(0,0.05,51)
peak_sens = np.zeros((51,))
peak_f = np.zeros((51,))
peak_bw = np.zeros((51,))
nopeak_sens = np.zeros((51,))
nopeak_f = np.zeros((51,))

# Vary parameter, measure peak sensitivity, peak frequency, peak FWHM (bandwidth)
for i, itmT in enumerate(vary_itmT):
    peak_power, prmT = find_optimal_prmT(itmT)
    kat = finesse.Model()
    kat.parse(
        f"""
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1
    var itmT {itmT}
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 500
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45
    
    # Check power in PRC
    #pd reflPRC prm.p1.o
    #pd circPRC bs.p1.i
    #pd trnsPRC bs.p3.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx
    
    m itmxar T=1 L=0 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm
    
    m etmx T=5u L=40u phi=179.99999
    
    # Check power in ARM X cavity
    #pd reflX itmx.p1.o
    #pd circX etmx.p1.i
    #pd trnsX etmx.p2.o

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy
    
    m itmyar T=1 L=0 phi=90
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm
    
    m etmy T=5u L=40u phi=90.00001
    
    # Check power in ARM Y cavity
    #pd reflY itmy.p1.o
    #pd circY etmy.p1.i
    #pd trnsY etmy.p2.o

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M
    
    ###########################################################################
    ###   vSRM
    ###########################################################################
    
    #m srm T=0.2 L=37.5u phi=-90
    
    s src bs.p4 SRC_BS.p1 L=354
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L=0
    m vSRM1 T=0 L=0 phi=-83.6435
    s vSRC2  SRC_BS.p3 vSRM2.p1 L=0
    m vSRM2 T=0 L=0 phi=-6.3565
    
    # Check power in SRC
    #pd reflSRC bs.p2.o
    #pd circSRC SRC_BS.p1.i
    #pd trnsSRC SRC_BS.p3.o
   
    ###########################################################################
    ###   Output & squeezing
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    #sq sqz db=-10 angle=90
    #link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    #qshot NSR_without_RP AS.p1.i nsr=True
    #pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 500, 10k, 200)
    
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP'])
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size == 0: # If no peak found, the max sensitivity and its frequency (lower bound is 500Hz) is recorded
        nopeak_sens[i] = np.min(np.abs(out['NSR_with_RP']))
        nopeak_f[i] = fsig[np.argmin(np.abs(out['NSR_with_RP']))]
        peak_sens[i] = np.nan
        peak_f[i] = np.nan
        peak_bw[i] = np.nan
        continue
    else:
        nopeak_sens[i] = np.nan
        nopeak_f[i] = np.nan
        
    fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
    left_idx = fwhm_idxs[2][0]
    right_idx = fwhm_idxs[3][0]
    interp_fsig = interpolate.interp1d(np.arange(201), fsig)
    peak_sens[i] = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
    peak_f[i] = fsig[peak_idxs[0]]
    peak_bw[i] = interp_fsig(right_idx) - interp_fsig(left_idx)
    
fig_peak_sens = go.Figure()
fig_peak_sens.add_trace(go.Scatter(x=vary_itmT, y=peak_sens,mode='lines+markers',name='Peak'))
fig_peak_sens.add_trace(go.Scatter(x=vary_itmT, y=nopeak_sens,mode='lines+markers',name='No peak'))
fig_peak_sens.update_layout(title="Peak Sensitivity vs. ITM T",xaxis_title="ITM T",yaxis_title="Peak Sensitivity [1/rt Hz]")
fig_peak_sens.show()

fig_peak_f = go.Figure()
fig_peak_f.add_trace(go.Scatter(x=vary_itmT, y=peak_f,mode='lines+markers',name='Peak'))
fig_peak_f.add_trace(go.Scatter(x=vary_itmT, y=nopeak_f,mode='lines+markers',name='No peak'))
fig_peak_f.update_layout(title="Peak Frequency vs. ITM T",xaxis_title="ITM T",yaxis_title="Peak Frequency [Hz]")
fig_peak_f.show()

fig_peak_bw = px.line(x=vary_itmT, y=peak_bw, markers=True, title="Peak Bandwidth vs. ITM T")
fig_peak_bw.update_xaxes(title_text="ITM T")
fig_peak_bw.update_yaxes(title_text="Peak Bandwidth [Hz]")
fig_peak_bw.show()

### Sensitivity Function  
Packages **Sensitivity Curve Interactive Dashboard** as a function. Input parameters and data is output as a dictionary including sensitivity curve data. Takes ~5s to evaluate. NOTE: prmT is swept at a resolution of 0.01 from 0.01 to 0.99 to maximise arm cavity power (X arm - power is matched with Y arm closely). Peaks are found using scipy findpeaks using the negative NSR curve (with RP). The left and right half maximum and hence FWHM are calculated by interpolating the xaxis (log spaced). vSRM is held on dark fringe and common and differential mode offsets tune from this setpoint.

In [20]:
# IN: diff, comm, srcL, vsrmL, vsrmT, itmT <- params
# OUT: Dictionary of dictionaries listed below as KEY1 (KEY2)
# xaxis
# prmT <- Optimised to maximise arm cavity power
# power_data (armX, armY, laser, onBS, PRC, SRC)
# peak_data (sens, freq, fwhm, lhm_f, rhm_f) <- l/rhm = left/right half max
# curve_data (NSR_with_RP, NSR_without_RP, signal)

import finesse
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

def run_config(diff,comm,srcL,vsrmL,vsrmT,itmT): 
    # Adapts from SRC_tunability_DCreadout_changedBW_fin3exp
    fsig = np.geomspace(1e3,10e3,151)
    peak_sens = np.nan
    peak_f = np.nan
    peak_bw = np.nan
    left_f = np.nan
    right_f = np.nan
    
    diff = float(diff)
    comm = float(comm)
    srcL = float(srcL)
    vsrmL = float(vsrmL)
    vsrmT = float(vsrmT)
    itmT = float(itmT)    
    prmT = find_optimal_prmT(diff,comm,srcL,vsrmL,vsrmT,itmT)
    
    kat = finesse.Model()
    kat.parse(
        f"""
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1
    var itmT {itmT}
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 500
    
    # CHECK Input laser power
    pd P_in L0.p1.o
    
    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45
    
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx
    
    m itmxar T=1 L=0 phi=180
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm
    
    m etmx T=5u L=40u phi=179.99999
    
    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy
    
    m itmyar T=1 L=0 phi=90
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm
    
    m etmy T=5u L=40u phi=90.00001
    
    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i
    
    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M
    
    ###########################################################################
    ###   vSRM
    ###########################################################################
    
    #m srm T=0.2 L=37.5u phi=-90
    
    s src bs.p4 SRC_BS.p1 L={srcL}
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L={vsrmL}
    m vSRM1 T={vsrmT} L=0 phi={-90+comm+diff}
    s vSRC2  SRC_BS.p3 vSRM2.p1 L={vsrmL}
    m vSRM2 T={vsrmT} L=0 phi={0+comm-diff}
    
    # CHECK SRC power
    pd P_SRC SRC_BS.p1.i
   
    ###########################################################################
    ###   Output & squeezing
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    #sq sqz db=-10 angle=90
    #link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    qshot NSR_without_RP AS.p1.i nsr=True
    pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 1k, 10k, 150)
    
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP']) # Take negative sensitivity to use findpeaks
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size != 0: # If a peak is found (first peak taken)
        fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
        left_idx = fwhm_idxs[2][0]
        right_idx = fwhm_idxs[3][0]
        interp_fsig = interpolate.interp1d(np.arange(151), fsig)
        
        left_f = interp_fsig(left_idx)
        right_f = interp_fsig(right_idx)
        peak_sens = np.real(out['NSR_with_RP'])[peak_idxs[0]]
        peak_f = fsig[peak_idxs[0]]
        peak_bw = right_f - left_f
        
    peak_dict = {"sens": peak_sens, "freq": peak_f, "fwhm": peak_bw, "lhm_f": left_f,"rhm_f": right_f}
    power_dict = {"armX": np.max(np.abs(out['P_armX'])), 
              "armY": np.max(np.abs(out['P_armY'])), "laser": np.max(np.abs(out['P_in'])), 
              "onBS": np.max(np.abs(out['P_BS'])), "PRC": np.max(np.abs(out['P_PRC'])),
             "SRC": np.max(np.abs(out['P_SRC']))}
    curve_dict = {"NSR_with_RP": np.abs(out['NSR_with_RP']), "NSR_without_RP": np.abs(out['NSR_without_RP']),
                 "signal": np.abs(out['signal'])}
    output = {"xaxis": fsig, "prmT": prmT, "peak_data": peak_dict, "power_data": power_dict, "curve_data": curve_dict} 
        
    return output

# Auto-tune prmT to maximise arm cavity power (impedance matching)
def find_optimal_prmT(diff,comm,srcL,vsrmL,vsrmT,itmT):
    vary_prmT = np.linspace(0.01,0.99,99)
    circX = np.zeros((99,))

    for i, prmT in enumerate(vary_prmT):
        kat = finesse.Model()
        kat.parse(
                f"""
        ###########################################################################
        ###   Variables
        ###########################################################################
        var Larm 4000
        var Mtm  74.1
        var itmT {itmT}
        var lmichx 4.5
        var lmichy 4.45

        ###########################################################################
        ###   Input optics
        ###########################################################################
        l L0 500

        # CHECK Input laser power
        pd P_in L0.p1.o

        s l_in L0.p1 prm.p1
        # Power recycling mirror
        m prm T={prmT} L=2e-05 phi=90
        s prc prm.p2 bs.p1 L=53

        # Central beamsplitter
        bs bs T=0.5 L=0 alpha=45

        ###########################################################################
        ###   X arm
        ###########################################################################
        s lx bs.p3 itmxar.p1 L=lmichx

        m itmxar T=1 L=0 phi=180
        s ar_thick itmxar.p2 itmx.p1 L=0
        m itmx T=itmT L=20u phi=180
        s LX itmx.p2 etmx.p1 L=Larm

        m etmx T=5u L=40u phi=179.99999

        # CHECK X-arm cavity power
        pd P_armX etmx.p1.i

        pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
        pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

        ###########################################################################
        ###   Y arm
        ###########################################################################
        s ly bs.p2 itmyar.p1 L=lmichy

        m itmyar T=1 L=0 phi=90
        s ar_thicky itmyar.p2 itmy.p1 L=0
        m itmy T=itmT L=20u phi=90
        s LY itmy.p2 etmy.p1 L=Larm

        m etmy T=5u L=40u phi=90.00001

        pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
        pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

        ###########################################################################
        ###   vSRM
        ###########################################################################

        #m srm T=0.2 L=37.5u phi=-90

        s src bs.p4 SRC_BS.p1 L={srcL}
        bs SRC_BS T=0.5 L=0 alpha=45
        s vSRC1 SRC_BS.p2 vSRM1.p1 L={vsrmL}
        m vSRM1 T={vsrmT} L=0 phi={-90+comm+diff}
        s vSRC2  SRC_BS.p3 vSRM2.p1 L={vsrmL}
        m vSRM2 T={vsrmT} L=0 phi={0+comm-diff}

        noxaxis()

        """)
        out = kat.run()
        circX[i] = out['P_armX']

    peak_T = vary_prmT[np.argmax(circX)]
    return peak_T
